<img alt="Cover" src="./images/1.cover.png" width="2000" />

---

# What is it?

<font size="5">We are already familiar with object-oriented programming (OOP), but Kotlin also borrows concepts from functional programming (FP). FP is a programming paradigm where programs are constructed by **applying** and **composing functions**.</font>

<img alt="Cover" src="./images/2.png" width="2000" />

---

# Our approach

<font size="5">**Disclaimer:** There won’t be any deep math or Haskell examples in this lecture. We will look at what we consider to be the most important FP features that can be used in Kotlin</font>

---

# We already know that...

<font size="5">- In Kotlin you can pass functions as the arguments of other functions:</font>

In [2]:
fun foo(bar: () -> Unit): Unit {  }

<font size="5">- If a function’s last argument is a function, then it can be put outside the parentheses:</font>

In [3]:
fun baz(start: Int, end: Int, step: (Int) -> Unit): Unit { step(start) }
baz(23, 42) { println("Magnificent!") }

Magnificent!


<font size="5">- If a function’s only argument is a function, then parentheses can be omitted altogether:</font>

In [4]:
foo { println("Kotlin keeps on giving!") }

<font size="5">- Lambdas can be assigned to `val`s and reassigned in `var`s:</font>

In [5]:
var lambda1: (Int) -> Double = { r -> r * 6.28 }
val lambda2 = { d: Int -> 3.14 * d.toDouble().pow(2) }
lambda1 = lambda2

<font size="5">- Lambda expressions can be replaced with function syntax:</font>

In [6]:
val sum = fun(a: Int, b: Int): Int = a + b
val sum2 = { a:Int, b: Int -> a + b }

<font size="5">- Declaring functions inside functions is allowed:</font>

In [7]:
fun global() {
    fun local() { println("Hello from local!") }
    
    local()
}
global()

Hello from local!


---

# Higher order functions (HOFs)

<font size="5">Functions that take other functions as **arguments** are called higher order functions.</font>

<font size="5">In Kotlin you frequently encounter them when working with collections:</font>

In [8]:
listOf(1, 2, 3).partition { it % 2 == 0 }
// OR
listOf(1, 2, 3).partition { x -> x % 2 == 0 }

([2], [1, 3])

<font size="5">Everything Kotlin allows you do with functions, which means that “functions in Kotlin are first-class citizens.”</font>

<font size="5">In functional programming, functions are designed to be pure. In simple terms, this means they cannot have a state. Loops have an iterator index, which is a state, so say goodbye to _conventional_ loops.</font>

In [9]:
// This is a LISP program transcribed to Kotlin; nobody actually writes like this

fun sumIter(term: (Double) -> Double, a: Double, next: (Double) -> Double, b: Double): Double {
   fun iter(a: Double, acc: Double): Double = if (a > b) acc else iter(next(a), acc + term(a))
   return iter(a, 0.0)
}

fun integral(f: (Double) -> Double, a: Double, b: Double, dx: Double): Double {
   fun addDx(x: Double) = x + dx
   return dx * sumIter(f, (a + (dx / 2.0)), ::addDx, b)
}

<font size="5">Often in the context of FP it is necessary to operate with the following functions: `map`, `filter`, and `fold`.</font>

<font size="5">`map` allows us to perform a function over _each_ element in a collection:</font>

In [10]:
val list = listOf(1, 2, 3)
println(list.forEach { it * it })
println(list.map { it * it }) // [1, 4, 9]

kotlin.Unit
[1, 4, 9]


<font size="5">*!!* What is the main difference between `map` and `forEach`?</font>

<font size="5">You can compose the functions to perform both operations:</font>

In [11]:
val l = listOf(1, 2, 3)
println(l.map { it * it }.map { it + 1 }) // [2, 5, 10]

[2, 5, 10]


In [12]:
// Same with
println(l.map { it * it + 1 }) // [2, 5, 10]

[2, 5, 10]


<img alt="Cover" src="./images/3.png" width="2000" />

<font size="5">**NB**: to compose complex functions by default you can use sequences, but be careful.</font>

<font size="5">`filter` returns a list containing only elements that match a given predicate:</font>

In [13]:
val l = listOf(1, 2, 3)
println(l.filter { it % 2 == 0 }) // [2]

[2]


<font size="5">Our third important function, `fold`, creates a mutable accumulator, which is updated on each round of the for and returns one value:</font>

In [14]:
val l = listOf(1, 2, 3)
var acc = 0

println(l.fold(0) { acc, x -> acc + x }) // 6

6


<font size="5">You can implement the `fold` function for any type, for example, you can fold a tree into a string representation.</font>

<font size="5">There are also `right` and `left` folds. They are equivalent if the operation is associative: `(a ○ b) ○ c = a ○ (b ○ c)`, but in any other case they yield different results.</font>

In [15]:
val list = listOf(1, 2, 3)
println(list.fold(0) { acc, x -> acc + x }) // (((0 + 1) + 2) + 3) = 6 
println(list.foldRight(0) { x, acc -> acc + x }) // (1 + (2 + (3 + 0))) = 6

6
6


In [16]:
println("PWND".fold("") { acc, x -> "${acc}${acc}$x" }) // PPWPPWNPPWPPWND 
println("PWND".foldRight("") { x, acc -> "${acc}${acc}$x" })  // DDNDDNWDDNDDNWP

PPWPPWNPPWPPWND
DDNDDNWDDNDDNWP


<font size="5">Make sure to: be careful with the order of your lambdas’ arguments:</font>

In [17]:
println(list.fold(0) { acc, x -> acc - x }) // (((0 - 1) - 2) - 3) = -6

-6


In [18]:
println(list.foldRight(0) { x, acc -> acc - x }) // (-1 + (-2 + (0 - 3))) = -6

-6


In [19]:
println(list.foldRight(0) { acc, x -> acc - x }) // INCORRECT, (1 - (2 - (3 - 0))) = 2

2


<font size="5">Lambdas are not the only functions that can be passed as arguments to functions expecting other functions, as _references_ to already defined functions can be as well:</font>

In [20]:
fun isEven(x: Int) = x % 2 == 0

val isEvenLambda = { x: Int -> x % 2 == 0 }

In [21]:
// Same results, different calls:
println(list.partition { it % 2 == 0 })

([2], [1, 3])


In [22]:
println(list.partition(::isEven)) // function reference

([2], [1, 3])


In [23]:
println(list.partition(isEvenLambda)) // pass lambda by name

([2], [1, 3])


---

# Lazy computations

<font size="5">Consider the following code:</font>

In [24]:
fun <F> withFunction(
    number: Int, even: F, odd: F
): F = when (number % 2) {
    0 -> even
    else -> odd
}

In [25]:
withFunction(4, println("even"), println("odd"))

even
odd


<font size="5">What will be printed to the console?</font>

---

# Deferred computations

<font size="5">Consider the following code:</font>

In [26]:
fun <F> withLambda(
    number: Int, even: () -> F, odd: () -> F
): F = when (number % 2) {
    0 -> even()
    else -> odd()
}

In [27]:
withLambda(4, { println("even") }, { println("odd") })

even


---

# Operator overloading

<font size="5">Kotlin has extension functions that you can use to override operators, for example the `iterator`. That is, you do not need to create a new entity that inherits from the `Iterable` interface, as you would in OOP code.</font>

In [28]:
class MyIterable<T> : Iterable<T> { // you need access to the sources of MyIterable
    override fun iterator(): Iterator<T> {
        TODO("Not yet implemented")
    }
}

In [29]:
// VS

class A<T>
operator fun <T> A<T>.iterator(): Iterator<T> = TODO("Not yet implemented")

In [30]:
for (i in A<Int>()) {}

kotlin.NotImplementedError: An operation is not implemented: Not yet implemented

---

# One last thing...

<font size="5">Is this code correct?</font>

In [31]:
enum class Color {
    WHITE,
    AZURE,
    HONEYDEW
}

In [32]:
fun Color.getRGB() = when (this) {
   Color.WHITE -> "#FFFFFF"
   Color.AZURE -> "#F0FFFF"
   Color.HONEYDEW -> "F0FFF0"
}

<font size="5">What is about this example?</font>

In [33]:
sealed class Color

class WhiteColor: Color()
class AzureColor: Color()
class HoneydewColor: Color()

In [34]:
//fun Color.getRGB() = when (this) {
//   is WhiteColor -> "#FFFFFF"
//   is AzureColor -> "#F0FFFF"
//   is HoneydewColor -> "F0FFF0"
//}

<font size="5">Consider the following code:</font>

In [35]:
sealed class Color

class WhiteColor(val name: String): Color()
class AzureColor(val name: String): Color()
class HoneydewColor(val name: String): Color()

<font size="5">We have the common part in **all** classes and we know that these are the **only** possible subclasses. Let’s move this code into the base class.</font>

In [36]:
sealed class NewColor(val name: String)

class WhiteColor(name: String): NewColor(name)
class AzureColor(name: String): NewColor(name)
class HoneydewColor(name: String): NewColor(name)

<font size="5">Actually, we have **equivalent** classes, i.e. _each_ function for the first version can be rewritten as the _second_ one.</font>

<img alt="Cover" src="./images/4.png" width="2000" />

<font size="5">In the first function, we have smart casts, but in the second one we don't have them.</font>

<font size="5">**Math time!** We can actually rewrite this in math terms:</font>

<font size="5">`WhiteColor * String + ... + HoneydewColor * String ≃ String * (WhiteColor + ... + HoneydewColor)`</font>

<font size="5">This is possible because we are actually operating with **algebraic data types** and can use their properties.</font>

<font size="5">This is not entirely true, but for most cases with sealed classes it works.</font>

---

# Final thought

<font size="5">FP in Kotlin does not kill OOP. Each of the concepts brings its own advantages and disadvantages, and it is important to combine them in order to get concise, readable and understandable code!</font>

<font size="5">If you are interested in the topic of FP in Kotlin for a more detailed study, come here: https://arrow-kt.io/</font>


<img alt="Cover" src="./images/5.png" width="2000" />